In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
with_mask = np.load('with_mask.npy')
without_mask = np.load('without_mask.npy')

In [3]:
with_mask

array([[[[  2, 254,   1],
         [ 19, 249,  13],
         [ 19, 250,  13],
         ...,
         [  9, 241,   6],
         [ 10, 242,   6],
         [ 10, 242,   6]],

        [[ 19, 250,  12],
         [208, 196, 141],
         [199, 191, 135],
         ...,
         [103, 102,  66],
         [105, 103,  65],
         [107, 105,  65]],

        [[ 18, 250,  12],
         [205, 197, 134],
         [200, 195, 133],
         ...,
         [ 98,  98,  57],
         [105, 104,  64],
         [105, 105,  66]],

        ...,

        [[ 15, 245,   9],
         [151, 135,  82],
         [140, 121,  76],
         ...,
         [143, 130,  89],
         [140, 130,  89],
         [141, 130,  90]],

        [[ 13, 244,   7],
         [142, 127,  78],
         [153, 134,  88],
         ...,
         [143, 129,  91],
         [140, 131,  91],
         [137, 126,  93]],

        [[ 16, 247,  10],
         [187, 175, 113],
         [192, 179, 112],
         ...,
         [131, 118,  79],
        

In [4]:
without_mask

array([[[[ 43, 248,  27],
         [100, 238,  67],
         [ 31, 165,  21],
         ...,
         [ 99, 236,  37],
         [ 96, 232,  38],
         [ 94, 231,  35]],

        [[100, 240,  65],
         [221, 216, 149],
         [ 70,  55,  50],
         ...,
         [215, 203,  76],
         [216, 203,  84],
         [209, 197,  72]],

        [[101, 241,  65],
         [206, 201, 132],
         [105,  90,  74],
         ...,
         [206, 194,  79],
         [215, 200,  80],
         [212, 197,  72]],

        ...,

        [[ 64, 210,  30],
         [ 43,  42,  16],
         [ 21,  19,  21],
         ...,
         [108, 111,  40],
         [ 83,  85,  27],
         [ 54,  58,  12]],

        [[ 55, 204,  29],
         [ 52,  53,  15],
         [ 29,  27,  13],
         ...,
         [109,  90,  34],
         [ 84,  73,  22],
         [ 54,  52,  11]],

        [[ 64, 213,  31],
         [120, 124,  74],
         [ 51,  43,  13],
         ...,
         [119, 105,  48],
        

In [3]:
with_mask.shape

(200, 50, 50, 3)

In [4]:
without_mask.shape

(200, 50, 50, 3)

In [5]:
with_mask = with_mask.reshape(200, 50*50*3)
without_mask = without_mask.reshape(200,50*50*3)

In [6]:
with_mask.shape

(200, 7500)

In [7]:
without_mask.shape

(200, 7500)

In [8]:
x = np.r_[without_mask, with_mask]           # r_ is used to concatinate the rows

In [9]:
x.shape

(400, 7500)

In [10]:
lables = np.zeros(x.shape[0])

In [11]:
lables[200:] = 1.0

In [12]:
name = {1:'withMask',0:'NO-Mask'}

In [13]:
# svm = support vector machine 
# svc = support vector classification

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [14]:
train_x, test_x, train_y, test_y = train_test_split(x,lables,test_size = 0.25, random_state = 42, stratify = lables)

In [15]:
train_x.shape

(300, 7500)

In [16]:
test_x.shape

(100, 7500)

In [17]:
train_y.shape

(300,)

In [18]:
test_y.shape

(100,)

In [19]:

# PCA - principal component analysis

from sklearn.decomposition import PCA
pca = PCA(n_components=3)
train_x = pca.fit_transform(train_x)


In [20]:
train_x.shape

(300, 3)

In [21]:
svm = SVC()
svm.fit(train_x,train_y)

SVC()

In [22]:
test_x = pca.transform(test_x)
y_pred = svm.predict(test_x)


In [23]:
accuracy_score(test_y, y_pred)

1.0

In [24]:
haar_data = cv2.CascadeClassifier('faceData.xml')
capture = cv2.VideoCapture(0)
data = []
font = cv2.FONT_HERSHEY_COMPLEX
while True:
    flag, img = capture.read()
    if flag:
        faces = haar_data.detectMultiScale(img)
        for x,y,w,h in faces:
            cv2.rectangle(img,(x,y),(w+x, h+y), (0,255,0),2)
            face = img[y:y+h, x:x+w,:]
            face = cv2.resize(face, (50,50))
            face = face.reshape(1,-1)
            face = pca.transform(face)
            pred = svm.predict(face)
            n = name[int(pred)]
            cv2.putText(img, n, (x,y), font, 2, (255,0,0),2)
        cv2.imshow('result',img)
        if cv2.waitKey(2) == 27:   #  ASCII value of Escape key is 27
            break
capture.release()
cv2.destroyAllWindows()